# Data

In [ ]:
%run nearest_mrt_station_to_bus_stops.ipynb

## Bus Routes Data

In [ ]:
from shapely.geometry import LineString

# combine bus routes
bus_routes_combined = bus_routes_gdf_3857.groupby(['ServiceNo', 'Direction']).apply(lambda x: LineString(x.geometry.tolist())).reset_index()
bus_routes_combined.columns = ['ServiceNo', 'Direction', 'geometry']
bus_routes_combined

## MRT Stations Data

In [ ]:
# mrt stations geodataframe
import os
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

mrt_stations_gdf = geopandas.read_file('./TrainStation_Jul2024/RapidTransitSystemStation.shp')
mrt_stations_gdf_3857 = mrt_stations_gdf.to_crs(3857)
mrt_stations_gdf_3857

In [ ]:
mrt_stations_gdf2 = geopandas.read_file('./mrt_stations/mrt_stations.shp')

mrt_stations_gdf2["lat_lng"] = geopandas.points_from_xy(mrt_stations_gdf2["LATITUDE"], mrt_stations_gdf2["LONGITUDE"])
mrt_stations_gdf2["line"] = mrt_stations_gdf2["CODE"].str.slice(0, 2) 

mrt_lines_code = ["NS", "EW", "CC", "NE", "TE", "DT"]

mrt_gdf_wo_lrt = mrt_stations_gdf2[mrt_stations_gdf2["line"].isin(mrt_lines_code)].reset_index()
mrt_gdf_wo_lrt = mrt_gdf_wo_lrt.drop("index", axis = 1)

mrt_gdf_wo_lrt["STN_NAM_DE"] = mrt_gdf_wo_lrt["BUILDING"].str.split('(').str[0].str.strip()
mrt_stations_3857_2 = mrt_gdf_wo_lrt.merge(mrt_stations_gdf_3857, how = "left", on = "STN_NAM_DE")

mrt_stations_3857_2

### Thomson-East Coast Line

In [ ]:
#brown_line = ['WOODLANDS NORTH MRT STATION',
            #   'WOODLANDS MRT STATION',
            #   'WOODLANDS SOUTH MRT STATION',
            #   'SPRINGLEAF MRT STATION',
            #   'LENTOR MRT STATION',
            #   'MAYFLOWER MRT STATION',
            #   'BRIGHT HILL MRT STATION',
            #   'UPPER THOMSON MRT STATION',
            #   'CALDECOTT MRT STATION',
            #   'STEVENS MRT STATION',
            #   'NAPIER MRT STATION',
            #   'ORCHARD BOULEVARD MRT STATION',
            #   'ORCHARD MRT STATION',
            #   'GREAT WORLD MRT STATION',
            #   'HAVELOCK MRT STATION',
            #   'OUTRAM PARK MRT STATION',
            #   'MAXWELL MRT STATION',
            #   'SHENTON WAY MRT STATION',
            #   'MARINA BAY MRT STATION',
            #   'GARDENS BY THE BAY MRT STATION',
            #   'TANJONG RHU MRT STATION',
            #   'KATONG PARK MRT STATION',
            #   'TANJONG KATONG MRT STATION',
            #   'MARINE PARADE MRT STATION',
            #   'MARINE TERRACE MRT STATION',
            #   'SIGLAP MRT STATION',
            #   'BAYSHORE MRT STATION',
            #   'BEDOK SOUTH MRT STATION',
            #   'SUNGEI BEDOK MRT STATION']

# # filter brown line stations from mrt stations geodataframe
# brown_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(brown_line)]

# # order brown line stations based on list
# brown_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(brown_line_stations_gdf['STN_NAM_DE'],
#                                                        categories = brown_line,
#                                                        ordered = True)
# brown_line_stations_gdf = brown_line_stations_gdf.sort_values('STN_NAM_DE')
# brown_line_stations_gdf

TEL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "TE"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
TEL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "TE"]

TEL_stations_gdf["order"] = pd.to_numeric(TEL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
TEL_stations_gdf = TEL_stations_gdf.sort_values('order')
TEL_stations_gdf




#### Route Visualisation

In [ ]:
import folium
import geopandas
from shapely.geometry import mapping
from shapely.geometry import LineString

# ordered list of station coordinates
TEL_stations_gdf['coordinates'] = TEL_stations_gdf['geometry'].centroid
TEL_stations_gdf

# create a linestring for the  route
TEL_route = LineString(TEL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
TEL_route_gdf = geopandas.GeoDataFrame(geometry = [TEL_route], crs = "EPSG:3857")
TEL_route_gdf_4326 = TEL_route_gdf.to_crs(4326)

# create base map
TEL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add TEL route layer
folium.GeoJson(
    data = mapping(TEL_route_gdf_4326),
    name = 'TEL',
    style_function = lambda x: {
        'color': 'brown',
        'weight': 4
    }
).add_to(TEL_map)

TEL_map


In [ ]:
# calculate the overlapping distance of each bus route and the TEL
buffer_distance = 150
buffered_TEL = TEL_route.buffer(buffer_distance)

overlap_distance_output_TEL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_TEL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_TEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_TEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Thomson-East Coast Line',
            'Intersection': overlap
            })

overlap_distance_TEL = geopandas.GeoDataFrame(overlap_distance_output_TEL)

# filter bus routes with overlap
bus_routes_overlap_TEL = overlap_distance_TEL[overlap_distance_TEL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_TEL

In [ ]:
bus_routes_overlap = bus_routes_overlap_TEL
bus_routes_overlap

### Downtown Line 

In [ ]:
# blue_line = ['SUNGEI KADUT MRT STATION',
#              'BUKIT PANJANG MRT STATION',
#              'CASHEW MRT STATION',
#              'HILLVIEW MRT STATION',
#              'HUME MRT STATION',
#              'BEAUTY WORLD MRT STATION',
#              'KING ALBERT PARK MRT STATION',
#              'SIXTH AVENUE MRT STATION',
#              'TAN KAH KEE MRT STATION',
#              'BOTANIC GARDENS MRT STATION',
#              'STEVENS MRT STATION',
#              'NEWTON MRT STATION',
#              'LITTLE INDIA MRT STATION',
#              'ROCHOR MRT STATION',
#              'BUGIS MRT STATION',
#              'PROMENADE MRT STATION',
#              'BAYFRONT MRT STATION',
#              'DOWNTOWN MRT STATION',
#              'TELOK AYER MRT STATION',
#              'CHINATOWN MRT STATION',
#              'FORT CANNING MRT STATION',
#              'BENCOOLEN MRT STATION',
#              'JALAN BESAR MRT STATION',
#              'BENDEMEER MRT STATION',
#              'GEYLANG BAHRU MRT STATION',
#              'MATTAR MRT STATION',
#              'MACPHERSON MRT STATION',
#              'UBI MRT STATION',
#              'KAKI BUKIT MRT STATION',
#              'BEDOK NORTH MRT STATION',
#              'BEDOK RESERVOIR MRT STATION',
#              'TAMPINES WEST MRT STATION',
#              'TAMPINES MRT STATION',
#              'TAMPINES EAST MRT STATION',
#              'UPPER CHANGI MRT STATION',
#              'EXPO MRT STATION']

# # filter blue line stations from mrt stations geodataframe
# blue_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(blue_line)]

# # order blue line stations based on list
# blue_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(blue_line_stations_gdf['STN_NAM_DE'],
#                                                        categories = blue_line,
#                                                        ordered = True)
# blue_line_stations_gdf = blue_line_stations_gdf.sort_values('STN_NAM_DE')
# blue_line_stations_gdf
DTL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "DT"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
DTL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "DT"]

DTL_stations_gdf["order"] = pd.to_numeric(DTL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
DTL_stations_gdf = DTL_stations_gdf.sort_values('order')
DTL_stations_gdf


#### Route Visualisation

In [ ]:
import folium
import geopandas
from shapely.geometry import mapping
from shapely.geometry import LineString
# ordered list of  station coordinates
DTL_stations_gdf['coordinates'] = DTL_stations_gdf['geometry'].centroid
DTL_stations_gdf

# create a linestring for the route
DTL_route = LineString(DTL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
DTL_route_gdf = geopandas.GeoDataFrame(geometry = [DTL_route], crs = "EPSG:3857")
DTL_route_gdf_4326 = DTL_route_gdf.to_crs(4326)

# create base map
DTL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add TEL route layer
folium.GeoJson(
    data = mapping(DTL_route_gdf_4326),
    name = 'DTL',
    style_function = lambda x: {
        'color': 'blue',
        'weight': 4
    }
).add_to(DTL_map)

DTL_map


In [ ]:
# calculate the overlapping distance of each bus route and the blue line
buffer_distance = 150
buffered_DTL = DTL_route.buffer(buffer_distance)

overlap_distance_output_DTL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_DTL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_DTL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Downtown Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_DTL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Downtown Line',
            'Intersection': overlap
            })

overlap_distance_DTL = pd.DataFrame(overlap_distance_output_DTL)
overlap_distance_DTL

# filter bus routes with overlap with DTL
bus_routes_overlap_DTL = overlap_distance_DTL[overlap_distance_DTL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_DTL

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_DTL, axis = 0)
bus_routes_overlap

### North-East Line 

In [ ]:
# purple_line = [
#     'HARBOURFRONT MRT STATION',
#     'OUTRAM PARK MRT STATION',
#     'CHINATOWN MRT STATION',
#     'CLARKE QUAY MRT STATION',
#     'DHOBY GHAUT MRT STATION',
#     'LITTLE INDIA MRT STATION',
#     'FARRER PARK MRT STATION',
#     'BOON KENG MRT STATION',
#     'POTONG PASIR MRT STATION',
#     'WOODLEIGH MRT STATION',
#     'SERANGOON MRT STATION',
#     'KOVAN MRT STATION',
#     'HOUGANG MRT STATION',
#     'BUANGKOK MRT STATION',
#     'SENGKANG MRT STATION',
#     'PUNGGOL MRT STATION'
# ]


# # filter purple line stations from mrt stations geodataframe
# purple_line_stations_gdf = mrt_stations_gdf_3857[mrt_stations_gdf_3857['STN_NAM_DE'].isin(purple_line)]

# # order purple line stations based on list
# purple_line_stations_gdf['STN_NAM_DE'] = pd.Categorical(purple_line_stations_gdf['STN_NAM_DE'],
#                                                        categories = purple_line,
#                                                        ordered = True)
# purple_line_stations_gdf = purple_line_stations_gdf.sort_values('STN_NAM_DE')
# purple_line_stations_gdf

NEL = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NE"]["STATION_NA"].tolist()

# filter stations from mrt stations geodataframe
NEL_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NE"]

NEL_stations_gdf["order"] = pd.to_numeric(NEL_stations_gdf["CODE"].str.slice(2, None))

# order stations based on list
NEL_stations_gdf = NEL_stations_gdf.sort_values('order')
NEL_stations_gdf

#### Route Visualisation

In [17]:
# ordered list of purple line station coordinates
NEL_stations_gdf['coordinates'] = NEL_stations_gdf['geometry'].centroid
NEL_stations_gdf

# create a linestring for the route
NEL_route = LineString(NEL_stations_gdf['coordinates'])

# convert from LineString to geodataframe
NEL_route_gdf = geopandas.GeoDataFrame(geometry = [NEL_route], crs = "EPSG:3857")
NEL_route_gdf_4326 = NEL_route_gdf.to_crs(4326)

# create base map
NEL_map = folium.Map(location = (1.359394, 103.814301), zoom_start = 12)

# add NEL route layer
folium.GeoJson(
    data = mapping(DTL_route_gdf_4326),
    name = 'NEL',
    style_function = lambda x: {
        'color': 'purple',
        'weight': 4
    }
).add_to(NEL_map)

NEL_map

In [ ]:
# calculate the overlapping distance of each bus route and the line
buffer_distance = 150
buffered_NEL = NEL_route.buffer(buffer_distance)

overlap_distance_output_NEL = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_NEL.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_NEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-East Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_NEL.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-East Line',
            'Intersection': overlap
            })

overlap_distance_NEL = pd.DataFrame(overlap_distance_output_NEL)
overlap_distance_NEL

# filter bus routes with overlap with NEL line
bus_routes_overlap_NEL = overlap_distance_NEL[overlap_distance_NEL['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_NEL

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_NEL], axis = 0)
bus_routes_overlap

# North-South Line (Red)

In [ ]:
red_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "NS"]["STATION_NA"].tolist()

# filter red line stations from mrt stations geodataframe
red_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "NS"]

red_line_stations_gdf["order"] = pd.to_numeric(red_line_stations_gdf["CODE"].str.slice(2, None))

# order red line stations based on list
red_line_stations_gdf = red_line_stations_gdf.sort_values('order')
red_line_stations_gdf

In [ ]:
# ordered list of red line station coordinates
red_line_stations_gdf['coordinates'] = red_line_stations_gdf['geometry_y'].centroid
red_line_stations_gdf


# create a linestring for the red line route
from shapely.geometry import LineString
red_line_route = LineString(red_line_stations_gdf['coordinates'])
red_line_route

In [ ]:
buffered_red_line = red_line_route.buffer(buffer_distance)

overlap_distance_output_red = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_red_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_red.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'North-South Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_red.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'North-South Line',
            'Intersection': overlap
            })

overlap_distance_red = pd.DataFrame(overlap_distance_output_red)
overlap_distance_red

In [ ]:
bus_routes_overlap_red = overlap_distance_red[overlap_distance_red['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_red

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_red], axis = 0)
bus_routes_overlap

# East-West Line (Green)

In [ ]:
green_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "EW"]["STATION_NA"].tolist()
green_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "EW"]

green_line_stations_gdf["order"] = pd.to_numeric(green_line_stations_gdf["CODE"].str.slice(2, None))

# order green line stations based on list
green_line_stations_gdf = green_line_stations_gdf.sort_values('order')
green_line_stations_gdf

In [ ]:
green_line_stations_gdf = geopandas.GeoDataFrame(green_line_stations_gdf, geometry='geometry_y')

green_line_stations_gdf['coordinates'] = green_line_stations_gdf['geometry_y'].centroid
green_line_stations_gdf

# create a linestring for the green line route
from shapely.geometry import LineString
green_line_route = LineString(green_line_stations_gdf['coordinates'])
green_line_route

In [ ]:
buffered_green_line = green_line_route.buffer(buffer_distance)

overlap_distance_output_green = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_green_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_green.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'East-West Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_green.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'East-West Line',
            'Intersection': overlap
            })

overlap_distance_green = pd.DataFrame(overlap_distance_output_green)
overlap_distance_green

In [ ]:
bus_routes_overlap_green = overlap_distance_green[overlap_distance_green['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_green

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_green], axis = 0)
bus_routes_overlap

# Circle Line (Yellow)

In [31]:
circle_line = mrt_stations_3857_2[mrt_stations_3857_2["line"] == "CC"]["STATION_NA"].tolist()

In [ ]:
circle_line_stations_gdf = mrt_stations_3857_2[mrt_stations_3857_2['line'] == "CC"]

circle_line_stations_gdf["order"] = pd.to_numeric(green_line_stations_gdf["CODE"].str.slice(2, None))

# order circle stations based on list
circle_line_stations_gdf = circle_line_stations_gdf.sort_values('order')
circle_line_stations_gdf

In [ ]:
circle_line_stations_gdf = geopandas.GeoDataFrame(circle_line_stations_gdf, geometry='geometry_y')

circle_line_stations_gdf['coordinates'] = circle_line_stations_gdf['geometry_y'].centroid
circle_line_stations_gdf

# create a linestring for the circle line route
from shapely.geometry import LineString
circle_line_route = LineString(circle_line_stations_gdf['coordinates'])
circle_line_route

In [ ]:
buffered_circle_line = circle_line_route.buffer(buffer_distance)

overlap_distance_output_circle = []

for i, route in tqdm(bus_routes_combined.iterrows()):
    overlap = buffered_circle_line.intersection(route.geometry)
    if overlap.is_empty:
        overlap_distance_output_circle.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': 0,
            'MRT line': 'Circle Line',
            'Intersection': overlap
            })
    else:
        overlap_distance_output_circle.append({
            'Bus Service': route['ServiceNo'],
            'Direction': route['Direction'],
            'Overlap Distance': overlap.length,
            'MRT line': 'Circle Line',
            'Intersection': overlap
            })

overlap_distance_circle = pd.DataFrame(overlap_distance_output_circle)
overlap_distance_circle

In [ ]:
bus_routes_overlap_circle = overlap_distance_circle[overlap_distance_circle['Overlap Distance'] > 0].sort_values(by = 'Overlap Distance', ascending = False)
bus_routes_overlap_circle

In [ ]:
bus_routes_overlap = pd.concat([bus_routes_overlap, bus_routes_overlap_circle], axis = 0)
bus_routes_overlap

# Bus Stops in Overlap

In [ ]:
# find the bus stops that belong to each intersection between the bus routes and the MRT lines
bus_routes_overlap_gdf_3857 = geopandas.GeoDataFrame(bus_routes_overlap,
                                                     geometry = bus_routes_overlap['Intersection'],
                                                     crs = "EPSG:3857")

overlap_count_output = []

for i, row in tqdm(bus_routes_overlap_gdf_3857.iterrows()):
    bus_no = row['Bus Service']
    direction = row['Direction']
    bus = bus_routes_gdf_3857[(bus_routes_gdf_3857['ServiceNo'] == bus_no) & (bus_routes_gdf_3857['Direction'] == direction)]
    for j, stop in bus.iterrows():
        if row['Intersection'].contains(stop.geometry):
            overlap_count_output.append({
                'Bus Service': bus_no,
                'Direction': direction,
                'Bus Stop Code': stop['BusStopCode'],
                'Description': stop['Description'],
                'MRT line': row['MRT line'],
                'geometry': stop.geometry
            })

overlap_bus_stops = geopandas.GeoDataFrame(overlap_count_output, crs = "EPSG:3857")


In [ ]:
overlap_bus_stops

In [39]:
# get the number of bus stops in the intersection between each bus route mrt line
overlap_count = overlap_bus_stops.groupby(['Bus Service', 'Direction', 'MRT line']).size().reset_index(name = 'Count').sort_values('Count', ascending = False)

bus_routes_overlap = bus_routes_overlap.merge(overlap_count, on = ['Bus Service', 'Direction', 'MRT line'], how = 'outer')
bus_routes_overlap = bus_routes_overlap[bus_routes_overlap['Count'].notna()]
bus_routes_overlap['Count'] = bus_routes_overlap['Count'].astype(int)

In [ ]:
bus_routes_overlap.sort_values(['Overlap Distance', 'Count'], ascending = False)